In [3]:
from ucimlrepo import fetch_ucirepo
import numpy as np

rice_cammeo_and_osmancik = fetch_ucirepo(id=545) 
X_1 = np.array(rice_cammeo_and_osmancik.data.features) 
y_1 = (np.array(rice_cammeo_and_osmancik.data.targets).T[0]== "Cammeo").astype(int)

In [61]:
import numpy as np

def soft_thresholding(a, b):
    return np.sign(a) * np.maximum(np.abs(a) - b, 0)

def sigmoid(x):
    return 1/(1+np.exp(-x))

class logisitic_regression():
    def __init__(self):
        pass
    
    def set_std_mean(self,X, epsilon = 1e-8):
        self.mean = np.mean(X, axis = 0)
        std = np.std(X,axis = 0)
        self.std = np.where(std > 0, std, epsilon)

    def standarize(self, X):
        return (X-self.mean)/self.std
    
    def fit(self, X, y, K=100, a = 1,weights = True, lambdas = None):
        X= np.array(X)
        n, p = X.shape
        y = np.array(y)
        self.set_std_mean(X)
        X = self.standarize(X)
        self.B= np.zeros(p)
        if not lambdas: 
            lambda_max= np.max(np.abs((y- 0.5)@X/n)) #since B = 0 w is 0.5 everywhere so this is the biggest possible value
            if a != 0:
                lambda_max /= a
            lambdas = np.logspace(np.log10(lambda_max), np.log10(0.001*lambda_max), K)
        for lambd in lambdas:
            for j in range(p):
                preds = sigmoid(X@self.B) 
                w = preds*(1-preds)
                xj = (X[:,j]).reshape((n,1))
                q=1/n
                wx2 = 1
                if weights:
                    #p and wx2 has different forms depends on version that we choose
                    q=w
                    wx2 = (w @ (xj**2))[0]
                sum = (q*w*X[:,j]*self.B[j] +q*(y-preds))@xj
                self.B[j] = soft_thresholding(sum[0],lambd*a)/(wx2 +lambd*(1-a))
        
    def predict(self, X):
        X = self.standarize(X)
        return np.round(1/(1+np.exp(-X@self.B)))           

In [67]:
lr_test = logisitic_regression()
lr_test.fit(X_1,y_1,a = 1, K=100,weights=True)
lr_test.B

array([ 0.21472809,  0.24689975,  0.26364419,  0.05469293,  0.19635691,
        0.2173956 , -0.02760829])

In [68]:
(y_1 == lr_test.predict(X_1)).mean()

0.916010498687664

In [303]:
import numpy as np

def soft_thresholding(a, b):
    return np.sign(a) * np.maximum(np.abs(a) - b, 0)

class logisitic_regression():
    def __init__(self):
        pass
    
    def set_std_mean(self,X, epsilon = 1e-8):
        self.mean = np.mean(X, axis = 0)
        std = np.std(X,axis = 0)
        self.std = np.where(std > 0, std, epsilon)

    def standarize(self, X):
        return (X-self.mean)/self.std

    def fit(self, X, y, a, epsilon = 0.1, K=100, weights = False, lambd = 0.1):
        X= np.array(X)
        n, p = X.shape
        y = np.array(y).reshape((n,1))
        #self.set_std_mean(X)
        #X = self.standarize(X)
        self.B = np.zeros((p, 1))
        for i in range(K):
            for j in range(p):
                preds = 1/(1+np.exp(-X@self.B))
                w =  preds*(1-preds)
                xj = X[:,j].reshape((n,1))
                # w_sumx2 = (w.T @ (X[:, j] ** 2))[0]
                # sum = xj.T@(w*xj*self.B[j,0] + y - preds)
                # self.B[j,0]= soft_thresholding(sum[0,0],lambd*a)/(w_sumx2+lambd*(1-a))
                num = xj.T@(w *xj * self.B[j,0]+ (y - preds))
                den = w.T@ (X[:, j] ** 2)
                self.B[j,0] = soft_thresholding(num[0,0] , lambd )/den[0]
                    
    def predict(self, X):
        return np.round(1/(1+np.exp(-X@self.B)))
        

In [304]:
lr_test = logisitic_regression()
lr_test.fit(X_1,y_1,a = 1, K=1)
lr_test.B

array([[-6.72101267e-06],
       [-1.70278080e-04],
       [ 1.39195647e-04],
       [-1.19711655e-03],
       [-2.60493747e-02],
       [ 1.38067288e-05],
       [-3.46227670e-01]])

In [306]:
lr_test = logisitic_regression()
lr_test.fit(X_1,y_1,a = 1, weights=True, K=100)
pred = lr_test.predict(X_1)
np.mean(y_1==pred)

0.513753694174055

In [292]:
1/(1+np.exp(-X_1@lr_test.B))

array([[0.98045316],
       [0.89453487],
       [0.83145026],
       ...,
       [0.42380265],
       [0.05094584],
       [0.02115381]])

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.special import expit  # Sigmoid function

def sigmoid(x):
    return 1/(1+np.exp(-x))

def soft_threshold(a, lambda_):
    """Funkcja soft-thresholding dla regularyzacji L1"""
    return np.sign(a) * np.maximum(np.abs(a) - lambda_, 0)

def cyclic_coordinate_descent(X, y, lambda_, max_iter=100, tol=1e-6):
    n, p = X.shape
    beta = np.zeros(p)
    
    for _ in range(max_iter):
        beta_old = beta.copy()
        
        for j in range(p):
            # Oblicz prawdopodobieństwa
            p_hat = sigmoid(X @ beta)
            w = p_hat * (1 - p_hat)  # Wagi dla IRLS
            
            # Obliczenie nowej wartości beta_j z soft-thresholding
            num = np.sum( (w *X[:, j] * beta[j]+ (y - p_hat)) * X[:, j])
            #print( ( (y - p_hat)) @ (X[:, j].reshape((n,1))))
            den = np.sum(w * X[:, j] ** 2)
            beta[j] = soft_threshold(num , lambda_ )/den
        
        # Sprawdzenie zbieżności
        if np.linalg.norm(beta - beta_old, ord=1) < tol:
            break
            
    return beta

# Przykładowe dane
np.random.seed(42)
X = np.random.randn(100, 5)
y = (np.random.rand(100) > 0.5).astype(int)  # Losowe etykiety 0/1
X = StandardScaler().fit_transform(X)  # Standaryzacja

# Uruchomienie algorytmu
lambda_ = 0.1
beta_estimated = cyclic_coordinate_descent(X_1, y_1, lambda_)
print("Oszacowane współczynniki beta:", beta_estimated)


[-1046507.]
[-33606.06106834]
[4732.41355787]
[-8491.62302494]
[-19.34648235]
[2207572.20684282]
[-146.30420862]
[2690320.29178998]
[-33723.29521518]
[4644.95159004]
[-8314.80157709]
[-18.00990966]
[2135257.9259353]
[-139.9011534]
[2575682.02370874]
[-32406.86183654]
[4562.38738421]
[-8144.68201171]
[-16.73706647]
[2065955.91370766]
[-133.79332763]
[2466133.37870124]
[-31170.36786653]
[4482.55182259]
[-7981.14450157]
[-15.52777141]
[1999651.99525284]
[-127.97550363]
[2361578.41882858]
[-30009.33889717]
[4405.42094949]
[-7824.03356346]
[-14.38108645]
[1936302.69452576]
[-122.44013368]
[2261888.76912523]
[-28919.15187607]
[4330.9559009]
[-7673.1651435]
[-13.29548721]
[1875841.53454549]
[-117.17796447]
[2166911.75129307]
[-27895.18269994]
[4259.1051683]
[-7528.33446193]
[-12.2690128]
[1818184.59184146]
[-112.17855526]
[2076477.31109182]
[-26932.91334809]
[4189.80682825]
[-7389.32254903]
[-11.29939363]
[1763235.269204]
[-107.43070403]
[1990403.80905465]
[-26028.00535534]
[4122.99066379]
[-